## Import libraries

In [ ]:
import pandas as pd
import numpy as np
from numba import njit
import ruptures as rpt
import matplotlib.pyplot as plt
from numexpr import evaluate
from scipy.cluster.vq import kmeans2
from typing import Tuple
from einops import rearrange

## Define util functions

In [ ]:
def generate_regular_grid(n: int) -> np.ndarray:
    """Generate a regular grid in the interval [0, 2*pi)^2.

    Parameters:
    - n (int): Number of points along each dimension.

    Returns:
    - np.ndarray: Regular grid of shape (n**2, 2).
    """
    x = np.linspace(0, 2 * np.pi, n, endpoint=False)
    grid = np.array(np.meshgrid(x, x)).T.reshape(-1, 2)
    return grid

In [ ]:
@njit
def get_best_previous_state_and_soc(
    soc_vec: np.ndarray, end_state: int, penalty: float
) -> Tuple[int, float]:
    """Update the sum of costs (soc) and the best previous state given the end state and a penalty.

    Parameters:
    - soc_vec (np.ndarray): Vector of sums of costs (soc), shape (n_states,).
    - end_state (int): The end state for which the update is performed.
    - penalty (float): Penalty value for the update.

    Returns:
    - Tuple[int, float]: Tuple containing the best previous state and the updated sum of costs.
    """
    n_states = soc_vec.shape[0]
    best_previous_state = end_state
    best_soc = soc_vec[best_previous_state]
    for k_state in range(n_states):
        if k_state != end_state:
            soc = soc_vec[k_state]
            if soc + penalty < best_soc:
                best_previous_state = k_state
                best_soc = soc + penalty
    return best_previous_state, best_soc


@njit
def get_state_sequence(costs: np.ndarray, penalty: float) -> np.ndarray:
    """Return the optimal state sequence for a given cost array and penalty.

    Parameters:
    - costs (np.ndarray): Array of cost values, shape (n_samples, n_states).
    - penalty (float): Penalty value.

    Returns:
    - np.ndarray: Optimal state sequence, shape (n_samples,).
    """
    n_samples, n_states = costs.shape
    soc_array = np.empty((n_samples + 1, n_states), dtype=np.float64)
    state_array = np.empty((n_samples + 1, n_states), dtype=np.int32)
    soc_array[0] = 0
    state_array[0] = -1

    # Forward loop
    for end in range(1, n_samples + 1):
        for k_state in range(n_states):
            best_state, best_soc = get_best_previous_state_and_soc(
                soc_vec=soc_array[end - 1], end_state=k_state, penalty=penalty
            )
            soc_array[end, k_state] = best_soc + costs[end - 1, k_state]
            state_array[end, k_state] = best_state

    # Backtracking
    end = n_samples
    state = np.argmin(soc_array[end])
    states = np.empty(n_samples, dtype=np.int32)
    while (state > -1) and (end > 0):
        states[end - 1] = state
        state = state_array[end, state]
        end -= 1
    return states

In [ ]:
def rle(inarray: np.ndarray):
    """run length encoding. Partial credit to R rle function.
    Multi datatype arrays catered for including non Numpy
    returns: tuple (runlengths, startpositions, values)"""
    ia = np.asarray(inarray)  # force numpy
    n = len(ia)
    if n == 0:
        return (None, None, None)
    else:
        y = ia[1:] != ia[:-1]  # pairwise unequal (string safe)
        i = np.append(np.where(y), n - 1)  # must include last element posi
        z = np.diff(np.append(-1, i))  # run lengths
        p = np.cumsum(np.append(0, z))[:-1]  # positions
        return (z, p, ia[i])

In [ ]:
@njit
def dist_func(x: np.ndarray, y: np.ndarray) -> float:
    # x, shape (2,)
    # y, shape (2,)
    diff = np.abs(x - y)
    return np.sum(np.fmin(diff, 2 * np.pi - diff))


@njit(parallel=True)
def compute_all_costs(signal, means):
    n_samples = signal.shape[0]
    n_states = means.shape[0]
    costs = np.empty((n_samples, n_states), dtype=np.float64)
    for k_state in range(n_states):
        for k_sample in range(n_samples):
            costs[k_sample, k_state] = dist_func(signal[k_sample], means[k_state])
    return costs

## Load data

In [ ]:
data_path = "/home/truong/code/geodesic-change/data/data_1.pkl.ikwt0u"
df = pd.read_pickle(data_path)
# all_signals, shape (n_signals, n_samples, 2)
all_signals = rearrange(
    df.to_numpy(),
    "n_samples (n_signals n_dims) -> n_signals n_samples n_dims",
    n_dims=2,
)
all_signals = np.deg2rad(all_signals)
all_signals[all_signals < 0] += 2 * np.pi

In [ ]:
interesting_signals[-10:]

In [ ]:
# a list of problematic signals
# fmt: off
interesting_signals = [473, 732, 623, 1092, 240, 146, 590, 250, 378, 629]
# interesting_signals = [506, 416, 1388, 259, 882, 138, 1203, 649, 1206, 1296, 456, 1063, 508, 972, 273, 417, 1115, 973, 38, 1291, 830, 224, 131, 1300, 223, 507, 26, 1341, 1190, 41, 1207, 504, 401, 305, 772, 957, 1156, 304, 837, 271, 725, 1191, 225, 770, 601, 958, 364, 492, 883, 1345, 134, 825, 1205, 365, 1238, 922, 274, 540, 413, 341, 958, 457, 1295, 40, 1340, 372, 1191, 737, 326, 597, 367, 413, 1229, 184, 1004, 1348, 507, 1070, 926, 1071, 602, 650, 368, 1063, 893, 1304, 306, 1059, 456, 372, 1116, 880, 307, 1349, 414, 660, 1062, 1258, 183, 361, 1333, 538, 42, 359, 1206, 592, 128, 225, 1041, 724, 409, 274, 1003, 1384, 1006, 1299, 831, 1060, 604, 1025, 360, 1100, 139, 725, 72, 693, 1296, 1237, 1360, 1346, 126, 371, 168, 998, 875, 1387, 1159, 455, 259, 923, 79, 1389, 139, 120, 1231, 741, 827, 1292, 1185, 740, 722, 767, 834, 407, 93, 1002, 362, 763, 1026, 427, 833, 135, 534, 838, 594, 1342, 1108, 258, 600, 1101, 1299, 879, 974, 408, 690, 1071, 1189, 1064, 593, 771, 1297, 490, 1359, 73, 301, 1298, 71, 1233, 978, 222, 830, 973, 826, 1005, 586, 834, 1235, 970, 838, 999, 132, 944, 41, 1244, 1011, 353, 1155, 72, 453, 485, 605, 419, 778, 1392, 1001, 652, 599, 777, 169, 194, 135, 477, 533, 1113, 428, 136, 826, 539, 219, 99, 1052, 1239, 771, 924, 768, 1259, 458, 1002, 434, 1064, 601, 1388, 1304, 711, 807, 819, 432, 373, 522, 694, 596, 1186, 360, 224, 1065, 366, 1067, 342, 565, 537, 307, 1300, 1066, 833, 829, 792, 1237, 1061, 1188, 873, 1061, 312, 726, 1192, 362, 1106, 559, 460, 1364, 1366, 26, 646, 943, 1385, 955, 331, 256, 300, 91, 634, 788, 828, 1390, 719, 1232, 885, 275, 1181, 366, 1340, 12, 1099, 597, 1236, 1351, 478, 598, 1301, 1264, 302, 1112, 820, 365, 70, 270, 689, 27, 956, 1189, 1239, 1352, 453, 74, 787, 650, 133, 1234, 723, 768, 545, 134, 399, 1066, 297, 1297, 829, 131, 1385, 11, 324, 133, 367, 835, 1240, 653, 793, 1059, 305, 94, 532, 996, 1234, 271, 127, 874, 25, 212, 420, 894, 773, 598, 784, 921, 452, 1177, 736, 797, 831, 1294, 1200, 1293, 332, 1334, 493, 136, 1301, 1202, 1345, 599, 919, 1292, 444, 959, 1185, 874, 832, 127, 956, 223, 368, 1157, 635, 773, 369, 458, 1303, 1285, 606, 1238, 603, 1112, 369, 539, 69, 560, 1339, 906, 1235, 187, 647, 1126, 1053, 868, 575, 181, 1067, 691, 1072, 883, 605, 1250, 408, 253, 922, 1305, 574, 1118, 952, 769, 1376, 445, 1203, 651, 1256, 245, 78, 38, 311, 322, 720, 798, 1298, 1211, 1190, 101, 1228, 1127, 832, 132, 737, 1109, 511, 952, 1210, 129, 772, 327, 322, 1156, 969, 66, 298, 710, 600, 1007, 175, 1000, 1243, 490, 839, 1144, 1295, 690, 894, 46, 1107, 137, 363, 1065, 745, 1101, 801, 1068, 299, 1100, 535, 1040, 1187, 130, 828, 910, 646, 593, 1393, 538, 641, 1098, 661, 492, 900, 68, 835, 501, 1211, 925, 410, 544, 1263, 745, 541, 1004, 951, 739, 1380, 14, 1256, 321, 734, 927, 69, 186, 531, 257, 886, 657, 1236, 342, 39, 64, 73, 74, 211, 486, 1143, 575, 1334, 1267, 187, 1031, 260, 383, 428, 867, 977, 1001, 876, 1360, 997, 130, 901, 803, 510, 946, 911, 719, 800, 744, 642, 1274, 435, 880, 1365, 128, 1082, 836, 121, 1158, 109, 1030, 884, 654, 1068, 75, 635, 1343, 491, 140, 1176, 336, 1186, 461, 1223, 480, 814, 540, 769, 302, 318, 1315, 1269, 402, 1069, 678, 414, 533, 1119, 278, 228, 808, 1140, 876, 765, 102, 788, 24, 1003, 137, 457, 166, 535, 997, 504, 1041, 1103, 802, 616, 988, 755, 1302, 35, 568, 269, 1273, 226, 1229, 967, 303, 1062, 489, 899, 1098, 536, 1294, 1342, 1367, 1005, 567, 68, 1007, 1268, 20, 1083, 228, 602, 594, 691, 927, 541, 596, 1099, 268, 222, 1160, 244, 1230, 503, 138, 337, 1269, 334, 1221, 1266, 569, 653, 735, 1006, 67, 1060, 349, 1255, 279, 218, 459, 551, 400, 36, 968, 978, 234, 487, 738, 1184, 895, 85, 22, 1010, 868, 898, 821, 970, 448, 108, 1127, 954, 1391, 308, 774, 337, 1302, 512, 633, 849, 1375, 200, 1023, 1033, 188, 252, 648, 617, 1337, 836, 46, 1012, 308, 924, 1201, 1372, 677, 1102, 954, 595, 545, 23, 689, 370, 429, 129, 1254, 1361, 1034, 723, 491, 512, 384, 355, 700, 1155, 1253, 570, 534, 1350, 290, 640, 486, 920, 881, 415, 923, 505, 279, 166, 182, 721, 65, 1393, 416, 335, 417, 181, 651, 882, 195, 907, 662, 79, 694, 879, 37, 764, 1245, 1254, 354, 227, 443, 306, 184, 656, 706, 774, 1118, 1293, 688, 712, 1286, 430, 190, 766, 169, 762, 1117, 233, 104, 221, 461, 24, 433, 803, 1158, 1035, 1386, 1391, 815, 649, 287, 925, 1209, 226, 180, 174, 1303, 1377, 1120, 1225, 713, 289, 363, 1109, 654, 469, 56, 14, 827, 776, 418, 524, 808, 150, 790, 652, 185, 523, 946, 275, 763, 104, 1157, 1045, 502, 1219, 945, 557, 1274, 60, 631, 45, 724, 1365, 1353, 20, 661, 953, 1379, 346, 172, 323, 310, 441, 57, 256, 296, 66, 1278, 1128, 641, 804, 777, 113, 933, 799, 100, 1054, 480, 1344, 878, 1257, 1204, 100, 343, 433, 521, 195, 1188, 482, 870, 753, 570, 779, 643, 75, 1010, 1161, 1242, 1154, 884, 467, 303, 520, 989, 757, 1160, 335, 488, 1032, 524, 895, 544, 1104, 1346, 77, 809, 1168, 754, 1222, 521, 953, 588, 201, 191, 957, 587, 1265, 1331, 1014, 1243, 1114, 1179, 399, 1172, 550, 103, 99, 361, 1113, 1132, 1178, 658, 1341, 902, 272, 1135, 255, 692, 632, 405, 1168, 348, 1170, 869, 173, 571, 182, 1244, 948, 564, 603, 1371, 423, 715, 429, 415, 743, 1159, 385, 105, 5, 109, 324, 639, 247, 1166, 220, 67, 122, 1363, 1187, 546, 1348, 1111, 887, 364, 167, 949, 1179, 888, 1058, 21, 877, 112, 987, 323, 610, 604, 1177, 638, 171, 888, 837, 782, 144, 920, 409, 789, 752, 640, 971, 1084, 1042, 102, 338, 1251, 702, 891, 939, 334, 1280, 1015, 1392, 1128, 336, 1368, 998, 1105, 633, 899, 781, 418, 431, 976, 1361, 1255, 1016, 1270, 80, 107, 488, 103, 810, 1277, 595, 693, 1247, 1354, 1338, 422, 254, 1367, 1248, 481, 319, 412, 802, 1121, 947, 905, 618, 439, 487, 987, 885, 1136, 740, 711, 568, 1036, 665, 1287, 1014, 1011, 935, 1017, 1008, 343, 55, 881, 944, 196, 1110, 695, 692, 227, 1394, 1268, 1009, 19, 704, 420, 901, 713, 421, 1358, 1373, 288, 406, 1347, 1316, 1362, 1220, 21, 523, 407, 662, 1317, 114, 151, 1083, 10, 1267, 1169, 312, 459, 1240, 536, 656, 993, 981, 92, 1114, 404, 1, 333, 436, 522, 1247, 851, 401, 546, 756, 42, 1134, 695, 229, 779, 98, 199, 1036, 1362, 896, 280, 926, 82, 1045, 1383, 645, 1131, 116, 384, 57, 340, 1316, 58, 151, 410, 947, 1125, 850, 398, 889, 986, 1111, 78, 1122, 1252, 375, 311, 1245, 551, 965, 608, 1336, 471, 1028, 655, 770, 203, 370, 183, 866, 714, 663, 823, 928, 141, 703, 412, 426, 955, 1350, 1105, 15, 636, 345, 385, 990, 937, 722, 1106, 576, 1351, 371, 1039, 1287, 709, 812, 304, 542, 54, 526, 1082, 167, 547, 325, 349, 257, 253, 101, 155, 13, 666, 1357, 1222, 841, 288, 190, 170, 995, 1119, 1272, 928, 989, 489, 578, 1266, 1394, 1355, 800, 313, 1178, 3, 850, 1162, 801, 460, 192, 1261, 548, 3, 1074, 478, 552, 921, 174, 1115, 537, 84, 479, 1070, 754, 1347, 1306, 785, 346, 1035, 611, 974, 476, 900, 168, 720, 941, 83, 291, 865, 333, 746, 795, 791, 1349, 434, 61, 909, 357, 1103, 579, 435, 238, 1136, 1207, 1212, 1133, 247, 1344, 979, 189, 655, 938, 1226, 1132, 1276, 193, 647, 1280, 806, 1116, 55, 188, 558, 549, 645, 714, 447, 201, 1386, 846, 1180, 1387, 1073, 778, 543, 291, 581, 824, 400, 23, 316, 1034, 878, 1013, 1129, 618, 430, 277, 147, 176, 1183, 300, 212, 1366, 313, 566, 473, 608, 196, 208, 789, 189, 402, 1180, 1076, 206, 1037, 712, 70, 1046, 643, 548, 248, 992, 229, 637, 76, 16, 843, 644, 1378, 173, 616, 617, 893, 438, 327, 455, 1220, 472, 309, 140, 1069, 1307, 718, 374, 1102, 775, 202, 1241, 936, 892, 270, 550, 1033, 948, 841, 421, 873, 840, 315, 314, 1305, 1108, 1210, 1281, 1214, 200, 634, 0, 1384, 321, 221, 642, 780, 469, 825, 197, 47, 1381, 759, 746, 988, 1322, 821, 567, 1024, 871, 573, 454, 582, 2, 708, 1359, 152, 482, 875, 1339, 1315, 9, 43, 193, 1358, 911, 1317, 1133, 979, 56, 258, 419, 839, 406, 669, 1072, 280, 1135, 246, 1286, 1371, 717, 756, 1124, 1369, 248, 668, 82, 1370, 65, 386, 1253, 811, 25, 146, 636, 249, 475, 1332, 613, 851, 513, 1249, 648, 282, 781, 729, 71, 1331, 1335, 454, 475, 1126, 990, 660, 1338, 142, 411, 1077, 569, 90, 4, 1330, 340, 532, 1021, 950, 1097, 1288, 145, 276, 1246, 81, 470, 982, 659, 152, 1307, 942, 909, 820, 1079, 1264, 639, 1012, 205, 1078, 587, 170, 1198, 525, 1137, 919, 320, 205, 246, 607, 1125, 117, 397, 822, 80, 240, 425, 22, 1017, 666, 16, 783, 999, 344, 511, 438, 592, 809, 508, 427, 1044, 386, 359, 477, 1330, 439, 426, 375, 449, 794, 9, 344, 784, 670, 992, 975, 204, 315, 13, 503, 1182, 214, 1374, 721, 1038, 85, 780, 531, 383, 356, 935, 496, 705, 1227, 670, 1161, 708, 289, 557, 194, 904, 210, 1138, 591, 1013, 632, 1182, 1212, 209, 671, 1250, 908, 175, 556, 12, 514, 980, 47, 872, 1230, 1369, 290, 509, 388, 766, 249, 963, 560, 281, 1121, 1134, 1097, 787, 215, 943, 15, 872, 1283, 1044, 1275, 1104, 1175, 606, 755, 963, 1224, 448, 515, 243, 1195, 31, 177, 844, 479, 398, 1309, 1378, 1223, 318, 905, 1144, 1281, 848, 115, 403, 866, 497, 142, 786, 1081, 612, 378, 1389, 63, 576, 962, 445, 1336, 1107, 555, 1374, 245, 637, 91, 817, 811, 726, 1085, 1306, 48, 1018, 710, 297, 776, 203, 1084, 1130, 869, 1032, 736, 865, 462, 715, 945, 914, 942, 153, 1221, 849, 1352, 1215, 37, 757, 903, 1382, 870, 165, 929, 1054, 493, 1335, 668, 443, 583, 468, 659, 938, 295, 1319, 110, 842, 126, 314, 1337, 204, 1395, 702, 912, 730, 351, 1214, 949, 1390, 749, 1042, 215, 1090, 908, 497, 793, 1312, 403, 579, 1213, 813, 1181, 1314, 437, 264, 27, 266, 1282, 498, 86, 1282, 373, 696, 111, 583, 1228, 969, 845, 239, 515, 1208, 374, 1046, 1380, 1370, 254, 577, 387, 856, 329, 619, 1353, 744, 282, 981, 235, 912, 278, 1050, 897, 1332, 1073, 1048, 30, 1277, 237, 552, 11, 1145, 317, 1265, 1022, 450, 582, 239, 959, 449, 903, 242, 1153, 1196, 422, 892, 1058, 380, 351, 619, 283, 1246, 476, 664, 350, 1381, 1323, 962, 89, 731, 934, 216, 2, 1318, 798, 840, 1249, 1120, 437, 6, 1197, 263, 431, 49, 716, 1290, 198, 341, 902, 94, 1048, 748, 556, 44, 543, 586, 667, 114, 1329, 84, 230, 185, 615, 991, 896, 1377, 58, 588, 584, 463, 1242, 1130, 49, 202, 915, 301, 141, 985, 709, 1333, 1027, 269, 864, 910, 30, 547, 267, 150, 1354, 472, 354, 176, 147, 1225, 1031, 355, 749, 347, 1141, 904, 260, 1055, 380, 1056, 664, 382, 1175, 748, 1311, 823, 1271, 767, 1321, 1023, 852, 107, 1226, 119, 122, 1328, 705, 483, 1043, 171, 118, 761, 554, 1096, 53, 108, 243, 404, 1363, 263, 6, 143, 81, 1000, 1146, 462, 1176, 732, 1137, 631, 481, 236, 513, 1233, 1285, 32, 729, 387, 1022, 117, 10, 1080, 867, 358, 1310, 1056, 805, 261, 1279, 1167, 1205, 436, 292, 562, 236, 446, 609, 527, 353, 688, 977, 432, 95, 310, 886, 822, 1232, 1117, 446, 613, 1202, 442, 328, 727, 1079, 753, 172, 1174, 687, 1153, 1215, 812, 686, 210, 913, 1329, 1165, 846, 1138, 1016, 824, 1379, 506, 790, 113, 1047, 265, 1026, 345, 1095, 1049, 377, 474, 1057, 217, 855, 1368, 209, 123, 148, 1142, 584, 1364, 750, 244, 516, 1171, 1291, 1147, 350, 1154, 499, 64, 699, 125, 941, 783, 1009, 574, 561, 716, 199, 1145, 610, 452, 1195, 1085, 964, 1279, 28, 442, 810, 186, 1382, 730, 960, 286, 1049, 281, 376, 1208, 1376, 213, 735, 50, 918, 549, 1086, 573, 332, 1227, 1167, 444, 799, 1375, 1057, 93, 1257, 542, 1322, 519, 121, 898, 1231, 214, 914, 216, 847, 1020, 1148, 964, 1278, 816, 88, 510, 1320, 1218, 669, 578, 45, 1192, 106, 1318, 447, 1088, 59, 50, 1321, 671, 116, 760, 1024, 1171, 440, 1081, 389, 1312, 976, 31, 468, 1313, 1218, 1308, 377, 90, 1258, 614, 638, 96, 179, 1259, 502, 118, 1040, 1248, 283, 213, 265, 1201, 775, 1146, 968, 1141, 268, 759, 913, 115, 683, 326, 1261, 262, 120, 1241, 144, 743, 687, 672, 53, 961, 1087, 48, 1131, 1076, 495, 1080, 762, 682, 381, 607, 60, 356, 1260, 580, 198, 397, 580, 211, 1, 1075, 782, 681, 996, 379, 1147, 1383, 498, 8, 581, 1343, 77, 1043, 293, 1209, 1309, 966, 728, 29, 1272, 1015, 747, 451, 530, 1025, 83, 994, 897, 317, 1193, 685, 36, 1027, 348, 286, 347, 792, 917, 761, 450, 701, 760, 1028, 620, 857, 54, 555, 1039, 329, 665, 742, 916, 658, 519, 528, 17, 44, 63, 701, 863, 35, 663, 309, 1263, 298, 585, 405, 165, 630, 764, 815, 325, 816, 526, 1194, 1251, 679, 1148, 1096, 319, 494, 856, 1053, 1140, 62, 853, 1328, 972, 985, 388, 125, 734, 96, 330, 1129, 667, 674, 1152, 553, 559, 266, 1055, 192, 328, 934, 241, 339, 565, 250, 255, 1075, 861, 220, 316, 1357, 887, 1151, 51, 915, 739, 681, 264, 853, 1197, 742, 1170, 62, 1030, 980, 509, 797, 566, 843, 1021, 441, 731, 795, 1327, 1086, 331, 678, 877, 806, 561, 1273, 1213, 496, 1260, 918, 644, 727, 684, 907, 747, 791, 871, 796, 680, 1089, 7, 609, 501, 119, 572, 43, 1373, 1149, 1074, 627, 562, 1283, 95, 273, 1184, 961, 112, 577, 863, 218, 262, 1029, 330, 19, 752, 1173, 1029, 352, 425, 163, 208, 589, 1095, 967, 485, 1124, 554, 299, 149, 1019, 864, 1200, 1216, 516, 917, 517, 563, 376, 891, 40, 76, 1194, 682, 1204, 817, 951, 1325, 704, 180, 1308, 179, 124, 92, 558, 819, 294, 1196, 852, 965, 1262, 5, 393, 1142, 87, 620, 197, 1150, 676, 937, 494, 396, 1020, 89, 514, 1319, 765, 758, 906, 529, 986, 1169, 677, 1219, 1193, 796, 847, 966, 1183, 818, 267, 495, 807, 960, 629, 238, 1143, 1262, 411, 676, 97, 1313, 178, 1276, 520, 1008, 158, 1052, 728, 699, 1018, 621, 1320, 842, 143, 936, 1252, 1139, 162, 991, 1047, 123, 707, 29, 391, 395, 862, 97, 718, 261, 975, 982, 284, 564, 794, 1110, 585, 855, 995, 161, 680, 854, 233, 390, 1372, 1224, 352, 1094, 1165, 741, 148, 234, 813, 657, 1139, 28, 235, 471, 381, 1093, 396, 818, 624, 930, 1174, 859, 505, 163, 860, 525, 733, 252, 971, 684, 614, 451, 1150, 39, 628, 1087, 679, 1275, 553, 848, 242, 994, 395, 1199, 983, 154, 862, 1356, 191, 287, 59, 1314, 154, 738, 1284, 251, 320, 615, 940, 950, 440, 88, 717, 424, 683, 758, 149, 675, 785, 500, 156, 52, 4, 466, 217, 159, 484, 382, 124, 1123, 1289, 703, 86, 98, 518, 34, 858, 1284, 1152, 814, 390, 916, 61, 499, 219, 110, 890, 932, 292, 207, 1326, 272, 18, 1050, 628, 993, 111, 470, 854, 686, 700, 673, 1123, 630, 1289, 87, 890, 685, 861, 530, 424, 563, 1019, 1051, 1356, 357, 389, 1327, 1094, 153, 1290, 697, 293, 1326, 1355, 1051, 105, 423, 237, 672, 207, 1037, 162, 1324, 624, 1122, 805, 889, 860, 157, 571, 786, 177, 1270, 358, 1038, 231, 161, 500, 285, 527, 844, 621, 1217, 804, 984, 1149, 733, 1163, 0, 17, 751, 591, 572, 484, 858, 1166, 1199, 1093, 1271, 611, 622, 483, 590, 1310, 295, 394, 1311, 276, 178, 294, 145, 157, 939, 277, 338, 845, 206, 473, 732, 623, 1092, 240, 146, 590, 250, 378, 629]
# fmt: on

## Plot one 2D signal

In [ ]:
signal_index = interesting_signals[-4]
signal = all_signals[signal_index]  # shape (n_samples, 2)
plt.plot(signal, alpha=0.5)

## Our method

1. Find discrete levels with a k-means algorithm.
2. Perform discrete change-point detection.

In [ ]:
# 1. Find discrete levels with a k-means algorithm.
centroids, _ = kmeans2(signal, k=30)
centroids[centroids > 2 * np.pi] -= 2 * np.pi
costs = compute_all_costs(signal, centroids)

In [ ]:
# 2. Perform discrete change-point detection.
penalty = 1000
states = get_state_sequence(costs, penalty)
approx = centroids[states]

In [ ]:
# Plot first dimension and its approximation
plt.plot(signal[:, 0])
plt.plot(approx[:, 0])

In [ ]:
# Plot second dimension and its approximation
plt.plot(np.unwrap(signal[:, 1]))
plt.plot(np.unwrap(approx[:, 1]))

In [ ]:
plt.plot(approx)
print("Lengths of successive segments: ", rle(states)[0])